In [ ]:
from collections import deque

def bfs_path(grid, start, goal):
    print("Grid:", grid)
    rows = len(grid)
    cols = len(grid[0])

    if not (0 <= start[0] < rows and 0 <= start[1] < cols):
        print("the start agent not on the map")
        return None
    if not (0 <= goal[0] < rows and 0 <= goal[1] < cols):
        print("the end agent not on the map")
        return None
    if grid[start[0]][start[1]] == 1 or grid[goal[0]][goal[1]] == 1:
        print("the agent starts in the wall.")
        return None

    visited = [[False]*cols for _ in range(rows)]
    parent = dict()

    queue = deque([start])
    visited[start[0]][start[1]] = True
    found = False

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while queue and not found:
        r, c = queue.popleft()
        if (r, c) == goal:
            found = True
            break

        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if not visited[nr][nc] and grid[nr][nc] == 0:
                    visited[nr][nc] = True
                    parent[(nr, nc)] = (r, c)
                    queue.append((nr, nc))

    if not found:
        print("not found")
        return None

    # Retrace the path
    path = []
    cur = goal
    while cur != start:
        path.append(cur)
        cur = parent[cur]
    path.append(start)
    path.reverse()
    return path


def test_bfs():
    # A 4x4 grid:
    #  0 = free cell
    #  1 = wall
    #
    #  Let's place a single wall at (1,1) and (1,2).
    #  BFS should find a path from (0,0) to (3,3).
    grid = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
    start = (0, 0)
    goal = (3, 3)

    path = bfs_path(grid, start, goal)
    print("Resulting path:", path)

test_bfs()



Grid: [[0, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0], [1, 0, 0, 0]]
Resulting path: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]
